In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import warnings

from datetime import datetime, timedelta
from IPython.display import display
# !pip install -U pandasql
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

warnings.filterwarnings("ignore")

In [17]:

# Extract the data

country_list = ["Bolivia", "Burundi", "Canada", "Chad", "Indonesia", "Mexico", "Niger", "Philippines", "USA"]
begin_date = 2005
end_date = 2020
nb_dates = end_date - begin_date + 1

attribute_names = ["Maternal_Leave_Benefits",
                    "Access_To_Basic_Drinking_Water_Total",
                    "Access_To_Basic_Drinking_Water_Urban",
                    "Access_To_Basic_Sanitation_Total",
                    "Access_To_Basic_Sanitation_Urban",
                    "Access_To_Managed_Drinking_Water_Total",
                    "Access_To_Managed_Sanitation_Total",
                    "Access_To_Basic_Handwashing_Facilities_Total",
                    "Access_To_Basic_Handwashing_Facilities_Urban",
                    "Unemployment_Female",
                    "Unemployment_Male",
                    "Unemployment_Total"
                    ]

nb_attributes = len(attribute_names)

QOL_data_raw = pd.read_csv("QOL_data_raw.csv")

QOL_transposed = QOL_data_raw.transpose()#flip dataframe

QOL_dimension = QOL_transposed.iloc[2,0:nb_attributes].transpose()#create data frame header columns
QOL_dimension = pd.DataFrame(QOL_dimension.values.reshape(1,-1))

count = 0
for k in range(len(country_list)):
    transposed_country = QOL_transposed.iloc[4+begin_date-1960:4+end_date-1960+1, count: count+nb_attributes]
    transposed_country.columns = range(transposed_country.columns.size)
    QOL_dimension = pd.concat([QOL_dimension, transposed_country], axis=0, ignore_index=True)
    count += nb_attributes


# add dates
date_table = pd.DataFrame(['Date'])
count = 0
for k in range(len(country_list)):
    date_table = pd.concat([date_table, pd.DataFrame([np.arange(begin_date, end_date+1)]).transpose()], axis=0)
date_table = date_table.reset_index().iloc[: , 1:]

QOL_dimension = pd.concat([date_table, QOL_dimension], axis=1, ignore_index=True)

QOL_dimension = QOL_dimension.drop(0)


country_table = pd.DataFrame(['Country'])
count = 0
for k in range(len(country_list)):
    country_table = pd.concat([country_table, pd.DataFrame([country_list[k]]*nb_dates)])
country_table = country_table.reset_index().iloc[: , 1:]

QOL_dimension = pd.concat([country_table, QOL_dimension], axis=1, ignore_index=True)

QOL_dimension = QOL_dimension.drop(0)


QOL_dimension.columns = ['Country_Ref', 'Date'] + attribute_names

QOL_dimension.insert(0, "QOL_Key",  np.arange(len(QOL_dimension)))


# Save it into a new csv file
QOL_dimension.to_csv('QOL_dimension.csv', index = False, header=True)

display(QOL_dimension.tail())


,QOL_Key,Country_Ref,Date,Maternal_Leave_Benefits,Access_To_Basic_Drinking_Water_Total,Access_To_Basic_Drinking_Water_Urban,Access_To_Basic_Sanitation_Total,Access_To_Basic_Sanitation_Urban,Access_To_Managed_Drinking_Water_Total,Access_To_Managed_Sanitation_Total,Access_To_Basic_Handwashing_Facilities_Total,Access_To_Basic_Handwashing_Facilities_Urban,Unemployment_Female,Unemployment_Male,Unemployment_Total
140,139,USA,2016,NaN,99.537139,99.842653,99.747554,99.841607,96.59864,98.311762,NaN,NaN,4.789,4.939,4.87
141,140,USA,2017,NaN,99.625816,99.864026,99.729968,99.839976,96.782652,98.299286,NaN,NaN,4.312,4.402,4.36
142,141,USA,2018,NaN,99.713121,99.885399,99.712774,99.838346,96.965188,98.287095,NaN,NaN,3.841,3.951,3.9
143,142,USA,2019,NaN,99.799044,99.906772,99.695994,99.836715,97.146243,98.275223,NaN,NaN,3.611,3.721,3.67
144,143,USA,2020,NaN,99.883527,99.928144,99.679619,99.835085,97.325718,98.263647,NaN,NaN,8.345,7.796,8.05


CSV for DB

In [18]:
final_table = QOL_dimension.drop(columns=["Country_Ref", "Date"])
display(final_table.tail(50))
final_table.to_csv("QOL_Dimension_DB.csv", index = False, header = True)

,QOL_Key,Maternal_Leave_Benefits,Access_To_Basic_Drinking_Water_Total,Access_To_Basic_Drinking_Water_Urban,Access_To_Basic_Sanitation_Total,Access_To_Basic_Sanitation_Urban,Access_To_Managed_Drinking_Water_Total,Access_To_Managed_Sanitation_Total,Access_To_Basic_Handwashing_Facilities_Total,Access_To_Basic_Handwashing_Facilities_Urban,Unemployment_Female,Unemployment_Male,Unemployment_Total
95,94,NaN,99.477264,100.0,91.736661,93.48725,42.938961,54.881722,89.843792,91.384273,3.517,3.457,3.48
96,95,NaN,99.679568,100.0,92.424063,93.946294,43.026285,57.341103,NaN,NaN,4.097,4.667,4.45
97,96,NaN,40.555573,90.904553,7.901512,29.027128,NaN,9.256827,7.123087,29.325238,2.711,3.414,3.1
98,97,NaN,41.190794,90.564554,8.401596,30.401364,NaN,9.814904,8.208276,29.93982,2.305,2.926,2.649
99,98,NaN,41.793852,90.224644,8.89362,31.795212,NaN,10.357996,9.293511,30.554401,1.901,2.444,2.203
100,99,NaN,42.364176,89.884822,9.377293,33.208673,NaN,10.885851,10.378542,31.168983,1.465,1.905,1.71
101,100,100.0,42.902902,89.545089,9.85317,34.641746,NaN,11.398954,11.463872,31.783564,1.064,1.429,1.267
102,101,NaN,43.409453,89.205443,10.320974,36.094432,NaN,11.897061,12.549247,32.398146,0.632,0.895,0.779
103,102,100.0,43.883283,88.865886,10.78038,37.56673,NaN,12.379905,13.634435,33.012728,0.219,0.399,0.32
104,103,NaN,44.325474,88.526417,11.232013,39.058641,NaN,12.848001,14.719907,33.627309,3.159,1.422,2.18
